Hilbert transformation

We use the Hilbert transform to determine the amplitude envelope and instantaneous frequency of an amplitude-modulated signal.
The amplitude envelope is given by magnitude of the analytic signal. The instantaneous frequency can be obtained by differentiating the
instantaneous phase in respect to time. The instantaneous phase corresponds to the phase angle of the analytic signal.

Results were used for the final analysis

In [1]:
import os
import sys
import matplotlib.pyplot as plt
import datajoint as dj
import neurodsp.filt
import numpy as np
import pandas as pd
import scipy
from pathlib import Path
import matplotlib.gridspec as gridspec

import warnings
warnings.filterwarnings("ignore")

import utils.helper_func as hf
import utils.preprocessing as preprocessing
import utils.plotting as plotting

config


Please enter DataJoint username:  root
Please enter DataJoint password:  ···············


Connecting root@localhost:3306
Epiphyte Setup


In [2]:
patient_id = 53
session_nr = 1

top_dir = '/home/anastasia/epiphyte/anastasia/output'
#top_dir =  '/media/alado/TOSHIBA_EXT/thesis/output'
folder = f'05-Hilbert_transform_{patient_id}'

df_patient_info = pd.read_csv(f'{top_dir}/{patient_id}_channel_info.csv')

fs = 32768
#The resulting sample rate is up / down times the original sample rate.
up = 1
down=32
fs_downs = (up/down)*fs
dt = 1/fs_downs      # sampling period/time/interval or time resolution, often denoted as T

signal_len = np.int(fs_downs*1.5)+1
times = np.linspace(-0.5, 1, signal_len)
times = times*1000
time_zero_idx = np.where(times == hf.find_nearest(times, 0))[0][0]
time_50_idx = np.where(times == hf.find_nearest(times, -50))[0][0]
time_100_idx = np.where(times == hf.find_nearest(times, -100))[0][0]

df_stim_info = pd.read_csv(f'{top_dir}/{patient_id}_df_stim_info.csv')

all_stim = np.unique(df_stim_info['stim_id'])
all_stim_name = np.unique(df_stim_info['stim_name'])
all_brain_areas = np.unique(df_patient_info['recording_site'])

filtering_types = ['theta', 'slow_gamma', 'fast_gamma']

epoch_start = 512

In [3]:
#check if save directories exist
for filtering_type in filtering_types:
    
    parent_dir = f'{top_dir}/{folder}/plots/all_filtering_together/db'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/plots/all_filtering_together/zscore'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/{filtering_type}/power/plots'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/{filtering_type}/power/raw'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/{filtering_type}/power/log'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/{filtering_type}/power/db'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/{filtering_type}/power/zscore'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/plots/all_filtering_together_prepost/db'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/plots/all_filtering_together_prepost/zscore'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
    parent_dir = f'{top_dir}/{folder}/plots/all_filtering_together_organized/zscore'
    if not os.path.exists(parent_dir):
        os.makedirs(parent_dir)
        hf.create_folder_structure(parent_dir, df_patient_info)
        
    #parent_dir = f'{top_dir}/{folder}/{filtering_type}/inst_freq'
    #if not os.path.exists(parent_dir):
    #    os.makedirs(parent_dir)
    #    hf.create_folder_structure(parent_dir, df_patient_info)

#hf.create_folder_structure(f'{top_dir}/05-Hilbert_transform/plots/all_filtering_together/', df_patient_info)      

In [4]:
#Hilbert transform for filtered signal
for filtering_type in filtering_types:
    print(filtering_type)
    save_dir = f'{top_dir}/{folder}/{filtering_type}'
    
    for i in range(len(df_patient_info['channel_name'])):
        ch = df_patient_info.loc[i,'channel_name']
        ch_site = df_patient_info.loc[i,'recording_site']
        epochs = np.load(f'{top_dir}/01-preprocessed_{patient_id}/epochs/{filtering_type}_2_sec/{patient_id}_epochs_channel-{ch}.npy')
        
        baseline_500_pre = []
        baseline_500_post = []     
        baseline_500_pre_log = []
        baseline_500_post_log = []   
        
        for st in all_stim:
            pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
            pre = pre.reset_index(drop=True)
            post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
            pre_index = np.array(pre['stim_index'])
            post_index = np.array(post['stim_index'])
            
            current_stim_index = pre.loc[0,'stim_id']
            current_stim_name = pre.loc[0,'stim_name']
            current_stim_paradigm = pre.loc[0,'paradigm']
            
            amplitude_envelope_pre, amplitude_envelope_post, instantaneous_frequency_pre, instantaneous_frequency_post, pre_epoch, post_epoch, phase_envelope_pre, phase_envelope_post = hf.make_hilbert_transform(epochs, pre_index, post_index, fs_downs)

            np.save(f'{save_dir}/power/raw/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_phase_envelope_pre.npy', phase_envelope_pre)
            np.save(f'{save_dir}/power/raw/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_phase_envelope_post.npy', phase_envelope_post)
            np.save(f'{save_dir}/power/raw/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_pre.npy', amplitude_envelope_pre)
            np.save(f'{save_dir}/power/raw/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_post.npy', amplitude_envelope_post)
            
            amplitude_envelope_pre_log = np.log10(amplitude_envelope_pre)
            amplitude_envelope_post_log = np.log10(amplitude_envelope_post)
            
            np.save(f'{save_dir}/power/log/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_pre.npy', amplitude_envelope_pre)
            np.save(f'{save_dir}/power/log/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_post.npy', amplitude_envelope_post)
            
            baseline_500_pre_log = np.average(amplitude_envelope_pre_log[:,:time_zero_idx], axis=1)
            baseline_500_post_log = np.average(amplitude_envelope_post_log[:,:time_zero_idx], axis=1)
            std_500_pre_log = np.std(amplitude_envelope_pre_log[:,:time_zero_idx], axis=1)
            std_500_post_log = np.std(amplitude_envelope_post_log[:,:time_zero_idx], axis=1) 
        
            #zscore
            amplitude_envelope_pre_zscore = ((amplitude_envelope_pre_log - baseline_500_pre_log[:,None])/std_500_pre_log[:,None])
            amplitude_envelope_post_zscore = ((amplitude_envelope_post_log - baseline_500_post_log[:,None])/std_500_post_log[:,None])
            np.save(f'{save_dir}/power/zscore/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_zscore_pre.npy', amplitude_envelope_pre_zscore)
            np.save(f'{save_dir}/power/zscore/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_zscore_post.npy', amplitude_envelope_post_zscore)                       
            
            #db
            #amplitude_envelope_pre_db = 10*np.log10(amplitude_envelope_pre/baseline_500_pre)
            #amplitude_envelope_post_db = 10*np.log10(amplitude_envelope_post/baseline_500_post)
            #np.save(f'{save_dir}/power/db/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_db_pre.npy', amplitude_envelope_pre_db)
            #np.save(f'{save_dir}/power/db/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_db_post.npy', amplitude_envelope_post_db)
        
# selecting 2 slices of data
#t1 = 600
#t2 = 650
#plt.figure(figsize=(6,6))
#plt.axhline(color='k', lw=1)
#plt.axvline(color='k', lw=1)
#plt.plot(analytic_signal.real[500:1500],analytic_signal.imag[500:1500], 'k.-', alpha=0.1, ms=2)
#plt.quiver(0,0,analytic_signal[t1].real,analytic_signal[t1].imag, angles='xy', scale_units='xy', scale=1, color='k')
#plt.quiver(0,0,analytic_signal[t2].real,analytic_signal[t2].imag, angles='xy', scale_units='xy', scale=1, color='g')
#plt.xlabel('Real');plt.ylabel('Imaginary')
#plt.box('off')
#inst_phase = instantaneous_phase[0]
#plt.title('Vector 1(k) phase: %.2f, vector 2(g) phase: %.2f'%(inst_phase[t1]/(2*np.pi)*360, inst_phase[t2]/(2*np.pi)*360));

theta
slow_gamma
fast_gamma


In [4]:
norm_type = 'zscore'
colors = ['#30398d', '#781C6D', '#ED6825']
colors_sem = ['#ebecf8', '#F6D8F2', '#FAD9C8']
#Hilbert transform for filtered signal    
for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    save_dir = f'{top_dir}/{folder}/plots/all_filtering_together/{norm_type}/{ch}_{ch_site}'
    
    for st in all_stim: 
        pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
        pre = pre.reset_index(drop=True)
        post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
        pre_index = np.array(pre['stim_index'])
        post_index = np.array(post['stim_index'])
        
        current_stim_index = pre.loc[0,'stim_id']
        current_stim_name = pre.loc[0,'stim_name']
        current_stim_paradigm = pre.loc[0,'paradigm']
        
        amplitude_envelope_pre = []
        amplitude_envelope_post = []
        pre_epochs = []
        post_epochs = []
        std_pre = []
        std_post = []
        
        for filtering_type in filtering_types:
            ampl_envelope_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_pre.npy')
            ampl_envelope_post = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_post.npy')
            std_pre.append(np.std(ampl_envelope_pre,axis=0))
            std_post.append(np.std(ampl_envelope_post,axis=0))            
            
            amplitude_envelope_pre.append(np.mean(ampl_envelope_pre, axis=0))
            amplitude_envelope_post.append(np.mean(ampl_envelope_post, axis=0))

            epochs = np.load(f'{top_dir}/01-preprocessed_{patient_id}/epochs/{filtering_type}_2_sec/{patient_id}_epochs_channel-{ch}.npy')
            pre_epoch = []
            post_epoch = []
            for j in range(len(pre_index)):
                pre_ep = epochs[pre_index[j],epoch_start:]
                post_ep = epochs[post_index[j],epoch_start:]
                pre_epoch.append(pre_ep)
                post_epoch.append(post_ep)
            pre_epochs.append(np.mean(pre_epoch, axis=0))
            post_epochs.append(np.mean(post_epoch, axis=0))
        
        fig, ((ax1, ax2), (ax3, ax4)) = plt.subplots(2, 2, sharey=False, figsize=(20, 10))
        title = f'Amplitude envelope and mean signal, channel: {ch}, recording_site: {ch_site}, stim_index:{current_stim_index}, stim_name:{current_stim_name}, paradigm:{current_stim_paradigm}'
        file_name = f"{{'amplitude_envelope_and_mean_signal_channel'!'{ch}', 'recording_site'!'{ch_site}', 'stim_index'!'{current_stim_index}', 'stim_name'!'{current_stim_name}'}}"
        fig.suptitle(title)
        
        for k in range(len(filtering_types)):
            ax1.plot(times, pre_epochs[k], label=filtering_types[k], color=colors[k])
        ax1.set_xlabel("Time (ms)")
        ax1.set_ylabel("Voltage (microvolts)")
        ax1.legend()
        ax1.set_title('Epoch pre-movie')
        ax1.axvline(x=0, color='k')
        
        #plot SEM = std/nsamples
        for k in range(len(filtering_types)):
            ax2.plot(times, amplitude_envelope_pre[k], label=filtering_types[k], color=colors[k])
            ax2.fill_between(times, amplitude_envelope_pre[k]+std_pre[k]/10, amplitude_envelope_pre[k]-std_pre[k]/10, color=colors_sem[k])
        ax2.set_xlabel("Time (ms)")
        ax2.set_ylabel(f"Power, {norm_type}")
        ax2.legend()
        ax2.set_title('Hilbert power pre-movie')
        ax2.axvline(x=0, color='k')

        for k in range(len(filtering_types)):
            ax3.plot(times, post_epochs[k], label=filtering_types[k], color=colors[k])
        ax3.set_xlabel("Time (ms)")
        ax3.set_ylabel("Voltage (microvolts)")
        ax3.legend()
        ax3.set_title('Epoch post-movie')
        ax3.axvline(x=0, color='k')
        
        for k in range(len(filtering_types)):
            ax4.plot(times, amplitude_envelope_post[k], label=filtering_types[k],color=colors[k])
            ax4.fill_between(times, amplitude_envelope_post[k]+std_post[k]/10, amplitude_envelope_post[k]-std_post[k]/10, color=colors_sem[k])
        ax4.set_xlabel("Time (ms)")
        ax4.set_ylabel(f"Power, {norm_type}")
        ax4.legend()
        ax4.set_title('Hilbert power post-movie')
        ax4.axvline(x=0, color='k')

        #plt.show()
        fig.savefig(f'{save_dir}/{file_name}.png', facecolor='white', transparent=False)
        plt.close()

In [5]:
#pre-post together
norm_type = 'zscore'
colors_post = ['#30398d', '#781C6D', '#ED6825']
colors = ['#767fd0', '#b82ba7', '#f39a6d']
colors_sem = ['#ebecf8', '#F6D8F2', '#FAD9C8']
#colors_sem = ['']
#Hilbert transform for filtered signal    
for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    save_dir = f'{top_dir}/{folder}/plots/all_filtering_together_prepost/{norm_type}/{ch}_{ch_site}'
    
    for st in all_stim: 
        pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
        pre = pre.reset_index(drop=True)
        post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
        pre_index = np.array(pre['stim_index'])
        post_index = np.array(post['stim_index'])
        
        current_stim_index = pre.loc[0,'stim_id']
        current_stim_name = pre.loc[0,'stim_name']
        current_stim_paradigm = pre.loc[0,'paradigm']
        
        amplitude_envelope_pre = []
        amplitude_envelope_post = []
        pre_epochs = []
        post_epochs = []
        std_pre = []
        std_post = []
        
        for filtering_type in filtering_types:
            ampl_envelope_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_pre.npy')
            ampl_envelope_post = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_post.npy')
            std_pre.append(np.std(ampl_envelope_pre,axis=0))
            std_post.append(np.std(ampl_envelope_post,axis=0))            
            
            amplitude_envelope_pre.append(np.mean(ampl_envelope_pre, axis=0))
            amplitude_envelope_post.append(np.mean(ampl_envelope_post, axis=0))

            epochs = np.load(f'{top_dir}/01-preprocessed_{patient_id}/epochs/{filtering_type}_2_sec/{patient_id}_epochs_channel-{ch}.npy')
            pre_epoch = []
            post_epoch = []
            for j in range(len(pre_index)):
                pre_ep = epochs[pre_index[j],epoch_start:]
                post_ep = epochs[post_index[j],epoch_start:]
                pre_epoch.append(pre_ep)
                post_epoch.append(post_ep)
            pre_epochs.append(np.mean(pre_epoch, axis=0))
            post_epochs.append(np.mean(post_epoch, axis=0))
        
        fig, (ax1, ax2) = plt.subplots(2, 1, sharey=False, figsize=(20, 10))
        title = f'Amplitude envelope and mean signal, channel: {ch}, recording_site: {ch_site}, stim_index:{current_stim_index}, stim_name:{current_stim_name}, paradigm:{current_stim_paradigm}'
        file_name = f"{{'amplitude_envelope_and_mean_signal_channel'!'{ch}', 'recording_site'!'{ch_site}', 'stim_index'!'{current_stim_index}', 'stim_name'!'{current_stim_name}'}}"
        fig.suptitle(title)
        
        for k in range(len(filtering_types)):
            ax1.plot(times, pre_epochs[k], label=f'{filtering_types[k]} pre', color=colors[k])
        for k in range(len(filtering_types)):
            ax1.plot(times, post_epochs[k], label=f'{filtering_types[k]} post', color=colors_post[k])      
        ax1.set_xlabel("Time (ms)")
        ax1.set_ylabel("Voltage (microvolts)")
        ax1.legend()
        ax1.set_title('Epoch')
        ax1.axvline(x=0, color='k')
        
        #plot SEM = std/nsamples
        for k in range(len(filtering_types)):
            ax2.plot(times, amplitude_envelope_pre[k], label=f'{filtering_types[k]} pre', color=colors[k])
            ax2.fill_between(times, amplitude_envelope_pre[k]+std_pre[k]/10, amplitude_envelope_pre[k]-std_pre[k]/10, color=colors_sem[k])
        for k in range(len(filtering_types)):
            ax2.plot(times, amplitude_envelope_post[k], label=f'{filtering_types[k]} post', color=colors_post[k])
            ax2.fill_between(times, amplitude_envelope_post[k]+std_post[k]/10, amplitude_envelope_post[k]-std_post[k]/10, color=colors_sem[k])     
        ax2.set_xlabel("Time (ms)")
        ax2.set_ylabel(f"Power, {norm_type}")
        ax2.legend()
        ax2.set_title('Hilbert power')
        ax2.axvline(x=0, color='k')

        #plt.show()
        fig.savefig(f'{save_dir}/{file_name}.png', facecolor='white', transparent=False)
        plt.close()

In [6]:
#pre-post together organized
norm_type = 'zscore'
colors_post = ['#30398d', '#781C6D', '#ED6825']
colors = ['#767fd0', '#b82ba7', '#f39a6d']
colors_sem = ['#ebecf8', '#F6D8F2', '#FAD9C8']
#colors_sem = ['']
#Hilbert transform for filtered signal    
for i in range(len(df_patient_info['channel_name'])):
    ch = df_patient_info.loc[i,'channel_name']
    ch_site = df_patient_info.loc[i,'recording_site']
    
    save_dir = f'{top_dir}/{folder}/plots/all_filtering_together_organized/{norm_type}/{ch}_{ch_site}'
    
    for m in range(len(all_stim_name)):
        all_st_type = df_stim_info.loc[df_stim_info['stim_name'] == all_stim_name[m]]
        all_st_type = np.unique(all_st_type['stim_id'])

        if len(all_st_type) == 1:
            fig = plt.figure(figsize=(20,4))
        elif len(all_st_type) == 4:
            fig = plt.figure(figsize=(20,13))
        elif len(all_st_type) == 6:
            fig = plt.figure(figsize=(20,16))
        else:
            fig = plt.figure(figsize=(20,12))
            
        outer = gridspec.GridSpec(len(all_st_type), 1, figure=fig, wspace=0.2, hspace=0.2)
        
        l=0
        for st in all_st_type:
            pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
            pre = pre.reset_index(drop=True)
            post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
            pre_index = np.array(pre['stim_index'])
            post_index = np.array(post['stim_index'])

            current_stim_index = pre.loc[0,'stim_id']
            current_stim_name = pre.loc[0,'stim_name']
            current_stim_paradigm = pre.loc[0,'paradigm']            

            amplitude_envelope_pre = []
            amplitude_envelope_post = []
            pre_epochs = []
            post_epochs = []
            std_pre = []
            std_post = []

            for filtering_type in filtering_types:
                ampl_envelope_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_pre.npy')
                ampl_envelope_post = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_post.npy')
                std_pre.append(np.std(ampl_envelope_pre,axis=0))
                std_post.append(np.std(ampl_envelope_post,axis=0))            

                amplitude_envelope_pre.append(np.mean(ampl_envelope_pre, axis=0))
                amplitude_envelope_post.append(np.mean(ampl_envelope_post, axis=0))

                epochs = np.load(f'{top_dir}/01-preprocessed_{patient_id}/epochs/{filtering_type}_2_sec/{patient_id}_epochs_channel-{ch}.npy')
                pre_epoch = []
                post_epoch = []
                for j in range(len(pre_index)):
                    pre_ep = epochs[pre_index[j],epoch_start:]
                    post_ep = epochs[post_index[j],epoch_start:]
                    pre_epoch.append(pre_ep)
                    post_epoch.append(post_ep)
                pre_epochs.append(np.mean(pre_epoch, axis=0))
                post_epochs.append(np.mean(post_epoch, axis=0))

            inner = gridspec.GridSpecFromSubplotSpec(1, 2, subplot_spec=outer[l], wspace=0.1, hspace=0.1)

            title = f'Amplitude envelope and mean signal, channel: {ch}, recording_site: {ch_site}, stim_index:{current_stim_index}, stim_name:{current_stim_name}, paradigm:{current_stim_paradigm}'
            file_name = f"{{'amplitude_envelope_and_mean_signal_channel'!'{ch}', 'recording_site'!'{ch_site}', 'stim_index'!'{current_stim_index}', 'stim_name'!'{current_stim_name}'}}"
            fig.suptitle(title)
            ax1 = fig.add_subplot(inner[0])
            ax2 = fig.add_subplot(inner[1])

            for k in range(len(filtering_types)):
                ax1.plot(times, pre_epochs[k], label=f'{filtering_types[k]} pre', color=colors[k])
            for k in range(len(filtering_types)):
                ax1.plot(times, post_epochs[k], label=f'{filtering_types[k]} post', color=colors_post[k])      
            ax1.set_xlabel("Time (ms)")
            ax1.set_ylabel("Voltage (microvolts)")
            ax1.legend()
            ax1.set_title('Epoch')
            ax1.axvline(x=0, color='k')

            #plot SEM = std/nsamples
            for k in range(len(filtering_types)):
                ax2.plot(times, amplitude_envelope_pre[k], label=f'{filtering_types[k]} pre', color=colors[k])
                ax2.fill_between(times, amplitude_envelope_pre[k]+std_pre[k]/10, amplitude_envelope_pre[k]-std_pre[k]/10, color=colors_sem[k])
            for k in range(len(filtering_types)):
                ax2.plot(times, amplitude_envelope_post[k], label=f'{filtering_types[k]} post', color=colors_post[k])
                ax2.fill_between(times, amplitude_envelope_post[k]+std_post[k]/10, amplitude_envelope_post[k]-std_post[k]/10, color=colors_sem[k])     
            ax2.set_xlabel("Time (ms)")
            ax2.set_ylabel(f"Power, {norm_type}")
            ax2.legend()
            ax2.set_title('Hilbert power')
            ax2.axvline(x=0, color='k')
            
            l=l+1
            
        #plt.show()
        fig.savefig(f'{save_dir}/{file_name}.png', facecolor='white', transparent=False)
        plt.close()

In [ ]:
#pre-post separately
norm_type = 'zscore'
colors_post = ['#30398d', '#781C6D', '#ED6825']
colors = ['#767fd0', '#b82ba7', '#f39a6d']
colors_sem = ['#ebecf8', '#F6D8F2', '#FAD9C8']
#colors_sem = ['']

#Hilbert transform for filtered signal     
for ft in range(len(filtering_types)):
    filtering_type = filtering_types[ft+2]
         
    for i in range(len(df_patient_info['channel_name'])):
        ch = df_patient_info.loc[i,'channel_name']
        ch_site = df_patient_info.loc[i,'recording_site']
        save_dir = f'{top_dir}/{folder}/{filtering_type}/power/plots/{ch}_{ch_site}'
        
        for st in all_stim: 
            pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
            pre = pre.reset_index(drop=True)
            post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
            pre_index = np.array(pre['stim_index'])
            post_index = np.array(post['stim_index'])

            current_stim_index = pre.loc[0,'stim_id']
            current_stim_name = pre.loc[0,'stim_name']
            current_stim_paradigm = pre.loc[0,'paradigm']
            
            ampl_envelope_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_pre.npy')
            ampl_envelope_post = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_post.npy')
            std_pre = np.std(ampl_envelope_pre,axis=0)
            std_post = np.std(ampl_envelope_post,axis=0)            
            
            amplitude_envelope_pre = np.mean(ampl_envelope_pre, axis=0)
            amplitude_envelope_post = np.mean(ampl_envelope_post, axis=0)

            epochs = np.load(f'{top_dir}/01-preprocessed_{patient_id}/epochs/{filtering_type}_2_sec/{patient_id}_epochs_channel-{ch}.npy')
            pre_epoch = []
            post_epoch = []
            for j in range(len(pre_index)):
                pre_ep = epochs[pre_index[j],epoch_start:]
                post_ep = epochs[post_index[j],epoch_start:]
                pre_epoch.append(pre_ep)
                post_epoch.append(post_ep)
            pre_epochs = np.mean(pre_epoch, axis=0)
            post_epochs = np.mean(post_epoch, axis=0)
        
            fig, (ax1, ax2) = plt.subplots(2, 1, sharey=False, figsize=(20, 10))
            title = f'Amplitude envelope and mean signal, channel: {ch}, recording_site: {ch_site}, stim_index:{current_stim_index}, stim_name:{current_stim_name}, paradigm:{current_stim_paradigm}'
            file_name = f"{{'amplitude_envelope_and_mean_signal_channel'!'{ch}', 'recording_site'!'{ch_site}', 'stim_index'!'{current_stim_index}', 'stim_name'!'{current_stim_name}'}}"
            fig.suptitle(title)
            
            for k in range(len(filtering_types)):
                ax1.plot(times, pre_epochs, label=f'{filtering_type} pre', color=colors[ft])
            for k in range(len(filtering_types)):
                ax1.plot(times, post_epochs, label=f'{filtering_type} post', color=colors_post[ft])        
            ax1.set_xlabel("Time (ms)")
            ax1.set_ylabel("Voltage (microvolts)")
            ax1.legend()
            ax1.set_title('Epoch')
            ax1.axvline(x=0, color='k')

            #plot SEM = std/nsamples
            for k in range(len(filtering_types)):
                ax2.plot(times, amplitude_envelope_pre, label=f'{filtering_type} pre', color=colors[ft])
                ax2.fill_between(times, amplitude_envelope_pre+std_pre/10, amplitude_envelope_pre-std_pre/10, color=colors_sem[ft])
            for k in range(len(filtering_types)):
                ax2.plot(times, amplitude_envelope_post, label=f'{filtering_type} post', color=colors_post[ft])
                ax2.fill_between(times, amplitude_envelope_post+std_post/10, amplitude_envelope_post-std_post/10, color=colors_sem[ft])      
            ax2.set_xlabel("Time (ms)")
            ax2.set_ylabel(f"Power, {norm_type}")
            ax2.legend()
            ax2.set_title('Hilbert power')
            ax2.axvline(x=0, color='k')

            #plt.show()
            fig.savefig(f'{save_dir}/{file_name}.png', facecolor='white', transparent=False)
            plt.close()

In [ ]:
#pre-post separately
norm_type = 'zscore'
colors_post = ['#30398d', '#781C6D', '#ED6825']
colors = ['#767fd0', '#b82ba7', '#f39a6d']
colors_sem = ['#ebecf8', '#F6D8F2', '#FAD9C8']
#colors_sem = ['']

#Hilbert transform for filtered signal     
for ft in range(len(filtering_types)):
    filtering_type = filtering_types[ft]
         
    for i in range(len(df_patient_info['channel_name'])):
        ch = df_patient_info.loc[i,'channel_name']
        ch_site = df_patient_info.loc[i,'recording_site']
        save_dir = f'{top_dir}/{folder}/{filtering_type}/power/plots/{ch}_{ch_site}'
        
        for st in all_stim: 
            pre = df_stim_info.loc[(df_stim_info['position']=='pre') & (df_stim_info['stim_id']==st)]
            pre = pre.reset_index(drop=True)
            post = df_stim_info.loc[(df_stim_info['position']=='post') & (df_stim_info['stim_id']==st)]
            pre_index = np.array(pre['stim_index'])
            post_index = np.array(post['stim_index'])

            current_stim_index = pre.loc[0,'stim_id']
            current_stim_name = pre.loc[0,'stim_name']
            current_stim_paradigm = pre.loc[0,'paradigm']
            
            ampl_envelope_pre = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_pre.npy')
            ampl_envelope_post = np.load(f'{top_dir}/{folder}/{filtering_type}/power/{norm_type}/{ch}_{ch_site}/{ch}_{ch_site}_{current_stim_index}_{current_stim_name}_amplitude_envelope_{norm_type}_post.npy')
            std_pre = np.std(ampl_envelope_pre,axis=0)
            std_post = np.std(ampl_envelope_post,axis=0)            
            
            amplitude_envelope_pre = np.mean(ampl_envelope_pre, axis=0)
            amplitude_envelope_post = np.mean(ampl_envelope_post, axis=0)

            epochs = np.load(f'{top_dir}/01-preprocessed_{patient_id}/epochs/{filtering_type}_2_sec/{patient_id}_epochs_channel-{ch}.npy')
            pre_epoch = []
            post_epoch = []
            for j in range(len(pre_index)):
                pre_ep = epochs[pre_index[j],epoch_start:]
                post_ep = epochs[post_index[j],epoch_start:]
                pre_epoch.append(pre_ep)
                post_epoch.append(post_ep)
            pre_epochs = np.mean(pre_epoch, axis=0)
            post_epochs = np.mean(post_epoch, axis=0)
        
            fig, (ax1, ax2) = plt.subplots(2, 1, sharey=False, figsize=(20, 10))
            title = f'Amplitude envelope and mean signal, channel: {ch}, recording_site: {ch_site}, stim_index:{current_stim_index}, stim_name:{current_stim_name}, paradigm:{current_stim_paradigm}'
            file_name = f"{{'amplitude_envelope_and_mean_signal_channel'!'{ch}', 'recording_site'!'{ch_site}', 'stim_index'!'{current_stim_index}', 'stim_name'!'{current_stim_name}'}}"
            fig.suptitle(title)
            
            for k in range(len(filtering_types)):
                ax1.plot(times, pre_epochs, label=f'{filtering_type} pre', color=colors[ft])
            for k in range(len(filtering_types)):
                ax1.plot(times, post_epochs, label=f'{filtering_type} post', color=colors_post[ft])        
            ax1.set_xlabel("Time (ms)")
            ax1.set_ylabel("Voltage (microvolts)")
            ax1.legend()
            ax1.set_title('Epoch')
            ax1.axvline(x=0, color='k')

            #plot SEM = std/nsamples
            for k in range(len(filtering_types)):
                ax2.plot(times, amplitude_envelope_pre, label=f'{filtering_type} pre', color=colors[ft])
                ax2.fill_between(times, amplitude_envelope_pre+std_pre/10, amplitude_envelope_pre-std_pre/10, color=colors_sem[ft])
            for k in range(len(filtering_types)):
                ax2.plot(times, amplitude_envelope_post, label=f'{filtering_type} post', color=colors_post[ft])
                ax2.fill_between(times, amplitude_envelope_post+std_post/10, amplitude_envelope_post-std_post/10, color=colors_sem[ft])      
            ax2.set_xlabel("Time (ms)")
            ax2.set_ylabel(f"Power, {norm_type}")
            ax2.legend()
            ax2.set_title('Hilbert power')
            ax2.axvline(x=0, color='k')

            #plt.show()
            fig.savefig(f'{save_dir}/{file_name}.png', facecolor='white', transparent=False)
            plt.close()